In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_attention_encoder_functions import (
    seqsignet_attention_encoder_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-10 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-10 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-10 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-10 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-10 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [49]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (10, 3, 5)]
num_layers = [1]
ffn_hidden_dim_sizes = [[32,32], [128,128], [512,512]]
dropout_rates = [0.1]
learning_rates = [5e-4, 3e-4, 1e-4, 1e-5]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [50]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "transformer_encoder_layers": 2,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [51]:
shift = 3
window_size = 5
n = 3

In [52]:
(
    seqsignet_attention_encoder_umap_kfold_11,
    best_seqsignet_attention_encoder_umap_kfold_11,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold_best_model.csv


In [53]:
seqsignet_attention_encoder_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.639272,0.379968,"[0.781563126252505, 0.17167381974248927, 0.186...",0.434493,"[0.68935041979673, 0.3341288782816229, 0.28]",0.385922,"[0.9022556390977443, 0.11551155115511551, 0.14]",None,0.646114,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.646360,0.357379,"[0.7850030731407499, 0.1751824817518248, 0.111...",0.449790,"[0.6827025871285012, 0.3333333333333333, 0.333...",0.369817,"[0.9233661075766338, 0.1188118811881188, 0.067...",None,0.653500,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.664368,0.429112,"[0.7986339488995698, 0.24816625916870413, 0.24...",0.499625,"[0.7097571942446043, 0.47877358490566035, 0.31...",0.425604,"[0.9129554655870445, 0.1674917491749175, 0.196...",None,0.656872,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.660536,0.343206,"[0.7958393807450411, 0.17963314358001262, 0.05...",0.486353,"[0.683991683991684, 0.38482384823848237, 0.390...",0.365890,"[0.951417004048583, 0.11716171617161716, 0.029...",None,0.648041,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.663793,0.342117,"[0.7957196104364556, 0.18656229615133724, 0.04...",0.483829,"[0.6810043218769294, 0.4454828660436137, 0.325]",0.366178,"[0.9569115095430885, 0.11798679867986799, 0.02...",None,0.648362,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.656130,0.457628,"[0.7870282054775856, 0.40312771503040834, 0.18...",0.487666,"[0.7441381087348622, 0.42568807339449544, 0.29...",0.450243,"[0.8351648351648352, 0.38283828382838286, 0.13...",None,0.652216,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.639847,0.412263,"[0.7738520576948525, 0.3403093721565059, 0.122...",0.467922,"[0.7133447182239571, 0.3793103448275862, 0.311...",0.410173,"[0.8455754771544245, 0.3085808580858086, 0.076...",None,0.651413,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.650383,0.293526,"[0.7871333964049195, 0.09344490934449096, 0.0]",0.322556,"[0.6658663465386154, 0.30180180180180183, 0.0]",0.339229,"[0.9624060150375939, 0.05528052805280528, 0.0]",None,0.642742,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.656322,0.269600,"[0.7931274669143256, 0.015673981191222573, 0.0]",0.272926,"[0.6625290923196276, 0.15625, 0.0]",0.332035,"[0.9878542510121457, 0.00825082508250825, 0.0]",None,0.641940,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [54]:
best_seqsignet_attention_encoder_umap_kfold_11["f1"].mean()

0.48906065803553905

In [55]:
best_seqsignet_attention_encoder_umap_kfold_11["precision"].mean()

0.4984881070870974

In [56]:
best_seqsignet_attention_encoder_umap_kfold_11["recall"].mean()

0.4833686372300234

In [57]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.77671022, 0.38878632, 0.30168544])

In [58]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_11["precision_scores"]).mean(
    axis=0
)

array([0.75120283, 0.42770682, 0.31655467])

In [59]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.8040293 , 0.3569857 , 0.28909091])

# history_length=20

In [20]:
shift = 3
window_size = 5
n = 6

In [21]:
(
    seqsignet_attention_encoder_umap_kfold_20,
    best_seqsignet_attention_encoder_umap_kfold_20,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_6_kfold_best_model.csv


In [22]:
seqsignet_attention_encoder_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.657088,0.447777,"[0.7888962326503635, 0.34957020057306587, 0.20...",0.495950,"[0.7265644022400779, 0.41496598639455784, 0.34...",0.436787,"[0.8629265471370735, 0.30198019801980197, 0.14...",None,0.655106,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.658238,0.451153,"[0.792107670194695, 0.312397596941562, 0.24895...",0.492555,"[0.7225268176400477, 0.4620355411954766, 0.293...",0.442952,"[0.8765182186234818, 0.23597359735973597, 0.21...",None,0.662492,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.660920,0.357443,"[0.7953527774416072, 0.17078052034689792, 0.10...",0.470371,"[0.6838709677419355, 0.445993031358885, 0.28125]",0.373775,"[0.9502602660497398, 0.10561056105610561, 0.06...",None,0.645793,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.662069,0.467303,"[0.7954790096878362, 0.36873563218390804, 0.23...",0.503355,"[0.7438349270256668, 0.4164070612668744, 0.349...",0.455229,"[0.8548293811451706, 0.33085808580858084, 0.18]",None,0.662653,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.661494,0.361055,"[0.7941531770959169, 0.17103448275862068, 0.11...",0.487406,"[0.6819502074688797, 0.5210084033613446, 0.259...",0.376408,"[0.9505494505494505, 0.10231023102310231, 0.07...",None,0.649005,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.661494,0.310448,"[0.7939401112557699, 0.1374045801526718, 0.0]",0.368108,"[0.67200961731116, 0.43231441048034935, 0.0]",0.350536,"[0.9699248120300752, 0.08168316831683169, 0.0]",None,0.643064,...,123,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.631226,0.389742,"[0.7726513301632182, 0.25222222222222224, 0.14...",0.427351,"[0.6953504510756419, 0.38605442176870747, 0.20...",0.389770,"[0.8692886061307113, 0.18729372937293728, 0.11...",None,0.609987,...,1,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.661111,0.318119,"[0.7935414935296212, 0.14305364511691884, 0.01...",0.495826,"[0.6731117824773414, 0.4297520661157025, 0.384...",0.353785,"[0.9664545980335454, 0.0858085808580858, 0.009...",None,0.645151,...,12,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.654598,0.323743,"[0.7891009482655144, 0.16103896103896104, 0.02...",0.456124,"[0.6745331418017648, 0.3780487804878049, 0.315...",0.354590,"[0.9505494505494505, 0.10231023102310231, 0.01...",None,0.641940,...,123,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.658238,0.469555,"[0.7922043010752688, 0.3573176700930005, 0.259...",0.494660,"[0.7400803616273229, 0.43922984356197353, 0.30...",0.459612,"[0.8522267206477733, 0.30115511551155116, 0.22...",None,0.660886,...,1,focal,2,True,5,Conv1d,None,concatenation,64,3


In [23]:
best_seqsignet_attention_encoder_umap_kfold_20["f1"].mean()

0.5099838614640518

In [24]:
best_seqsignet_attention_encoder_umap_kfold_20["precision"].mean()

0.5105735467322742

In [25]:
best_seqsignet_attention_encoder_umap_kfold_20["recall"].mean()

0.5102778484904592

In [26]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.77563718, 0.42640384, 0.32791057])

In [27]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_20["precision_scores"]).mean(
    axis=0
)

array([0.76934738, 0.4429996 , 0.31937366])

In [28]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.78214768, 0.41171617, 0.3369697 ])

# history_length=35

In [29]:
shift = 3
window_size = 5
n = 11

In [30]:
(
    seqsignet_attention_encoder_umap_kfold_35,
    best_seqsignet_attention_encoder_umap_kfold_35,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_11_kfold_best_model.csv


In [31]:
seqsignet_attention_encoder_umap_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.650766,0.396536,"[0.7854527086753378, 0.2581699346405229, 0.145...",0.482444,"[0.6971531046850482, 0.3798076923076923, 0.370...",0.395272,"[0.8993637941006362, 0.19554455445544555, 0.09...",None,0.656712,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.661494,0.426731,"[0.7933173016073916, 0.2651997683844818, 0.221...",0.497850,"[0.7053792482556831, 0.4446601941747573, 0.343...",0.419628,"[0.9063042220936958, 0.18894389438943895, 0.16...",None,0.654785,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.640613,0.422239,"[0.7810067462376753, 0.24831460674157302, 0.23...",0.459471,"[0.7082352941176471, 0.3890845070422535, 0.281...",0.419414,"[0.8704453441295547, 0.18234323432343233, 0.20...",None,0.648362,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.653065,0.371318,"[0.7842704887908857, 0.20927318295739347, 0.12...",0.476071,"[0.6803400637619553, 0.4348958333333333, 0.312...",0.379338,"[0.9256795835743205, 0.1377887788778878, 0.074...",None,0.651574,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.656513,0.329344,"[0.788654194327097, 0.19937694704049844, 0.0]",0.361314,"[0.6768178993163455, 0.4071246819338422, 0.0]",0.358926,"[0.9447657605552342, 0.132013201320132, 0.0]",None,0.648041,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.661111,0.367753,"[0.7941391941391942, 0.20711166562694944, 0.10...",0.483998,"[0.687235841081995, 0.42455242966751916, 0.340...",0.379131,"[0.940427993059572, 0.13696369636963696, 0.06]",None,0.650931,...,123,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.647701,0.284934,"[0.7873848334514528, 0.06741573033707866, 0.0]",0.297317,"[0.665535143769968, 0.22641509433962265, 0.0]",0.334485,"[0.9638519375361481, 0.039603960396039604, 0.0]",None,0.641940,...,1,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.666667,0.317205,"[0.7970191625266146, 0.15459610027855153, 0.0]",0.389958,"[0.6743394715772618, 0.4955357142857143, 0.0]",0.355282,"[0.9742625795257375, 0.09158415841584158, 0.0]",None,0.643064,...,12,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.616092,0.311654,"[0.761250766400981, 0.11365040148239655, 0.060...",0.353102,"[0.6608473493719396, 0.22604422604422605, 0.17...",0.336633,"[0.8976286871023713, 0.07590759075907591, 0.03...",None,0.629897,...,123,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.651724,0.492981,"[0.7863343601232147, 0.38648648648648654, 0.30...",0.500103,"[0.762214983713355, 0.4255952380952381, 0.3125]",0.488663,"[0.8120300751879699, 0.35396039603960394, 0.3]",None,0.663455,...,1,focal,2,True,5,Conv1d,None,concatenation,64,3


In [32]:
best_seqsignet_attention_encoder_umap_kfold_35["f1"].mean()

0.5009972195625266

In [33]:
best_seqsignet_attention_encoder_umap_kfold_35["precision"].mean()

0.5070925833312471

In [34]:
best_seqsignet_attention_encoder_umap_kfold_35["recall"].mean()

0.49704063138143395

In [35]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_35["f1_scores"]).mean(axis=0)

array([0.77759858, 0.41077495, 0.31461813])

In [36]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_35["precision_scores"]).mean(
    axis=0
)

array([0.75927255, 0.44158948, 0.32041572])

In [37]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_35["recall_scores"]).mean(axis=0)

array([0.79699248, 0.3850385 , 0.30909091])

# history_length=80

In [38]:
shift = 3
window_size = 5
n = 26

In [39]:
(
    seqsignet_attention_encoder_umap_kfold_80,
    best_seqsignet_attention_encoder_umap_kfold_80,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_26_kfold_best_model.csv


In [40]:
seqsignet_attention_encoder_umap_kfold_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.648276,0.427106,"[0.7837592424438967, 0.2873096446700507, 0.210...",0.487718,"[0.7106563161609033, 0.3733509234828496, 0.379...",0.417526,"[0.8736263736263736, 0.2334983498349835, 0.145...",None,0.651092,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.661111,0.320587,"[0.7920886452996544, 0.16967267869071478, 0.0]",0.373057,"[0.6735562310030395, 0.4456140350877193, 0.0]",0.355345,"[0.9612492770387507, 0.10478547854785479, 0.0]",None,0.643224,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.644061,0.354832,"[0.7793162602622227, 0.1848844472204872, 0.100...",0.440751,"[0.6761641505422071, 0.38046272493573263, 0.26...",0.367846,"[0.9196067090803933, 0.12211221122112212, 0.06...",None,0.651252,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.654789,0.284823,"[0.7911392405063291, 0.06332842415316642, 0.0]",0.319892,"[0.6651556957035869, 0.2945205479452055, 0.0]",0.337159,"[0.9759976865240023, 0.03547854785478548, 0.0]",None,0.644830,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.649234,0.303626,"[0.7862085421140539, 0.1246684350132626, 0.0]",0.328913,"[0.6691714053614947, 0.31756756756756754, 0.0]",0.343474,"[0.9528629265471371, 0.07755775577557755, 0.0]",None,0.643224,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.650383,0.349053,"[0.7847457627118644, 0.15699208443271767, 0.10...",0.457797,"[0.6749271137026239, 0.39144736842105265, 0.30...",0.366356,"[0.937246963562753, 0.09818481848184818, 0.063...",None,0.654303,...,123,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.640996,0.293090,"[0.7811943488068384, 0.06633020908435472, 0.03...",0.350165,"[0.6626384692849949, 0.26285714285714284, 0.125]",0.335851,"[0.951417004048583, 0.037953795379537955, 0.01...",None,0.635517,...,1,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.652107,0.296974,"[0.7880724174653886, 0.10284920083391244, 0.0]",0.330975,"[0.6669337071900661, 0.32599118942731276, 0.0]",0.341347,"[0.9629843840370156, 0.06105610561056106, 0.0]",None,0.641618,...,12,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.653065,0.290308,"[0.7892630091829528, 0.08166189111747851, 0.0]",0.325130,"[0.6656076250992852, 0.30978260869565216, 0.0]",0.338792,"[0.9693464430306535, 0.04702970297029703, 0.0]",None,0.643706,...,123,focal,2,True,5,Conv1d,None,concatenation,64,2
0,None,0.649808,0.515510,"[0.7740137850124652, 0.45984251968503936, 0.31...",0.514403,"[0.7851829812555787, 0.4397590361445783, 0.318...",0.517426,"[0.7631578947368421, 0.48184818481848185, 0.30...",None,0.657996,...,1,focal,2,True,5,Conv1d,None,concatenation,64,3


In [41]:
best_seqsignet_attention_encoder_umap_kfold_80["f1"].mean()

0.4946934941144896

In [42]:
best_seqsignet_attention_encoder_umap_kfold_80["precision"].mean()

0.5027473899107543

In [43]:
best_seqsignet_attention_encoder_umap_kfold_80["recall"].mean()

0.49242712819575774

In [44]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_80["f1_scores"]).mean(axis=0)

array([0.78077352, 0.41137528, 0.29193168])

In [45]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_80["precision_scores"]).mean(
    axis=0
)

array([0.76183202, 0.44827906, 0.29813109])

In [46]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_80["recall_scores"]).mean(axis=0)

array([0.80171583, 0.38283828, 0.29272727])

# history_length=110

In [47]:
shift = 3
window_size = 5
n = 36

In [48]:
(
    seqsignet_attention_encoder_umap_kfold_110,
    best_seqsignet_attention_encoder_umap_kfold_110,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 36: history length = 110
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
seqsignet_attention_encoder_umap_kfold_110

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_110["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_110["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_110["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_110["precision_scores"]).mean(
    axis=0
)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_110["recall_scores"]).mean(axis=0)